In [29]:
# import cv2 #opencv
# import os
# import time
# import uuid

In [30]:
import torch
import numpy as np

data_path_train = "../dataset/sign_mnist_train.csv"
data_path_test  = "../dataset/sign_mnist_test.csv"

arr_train = np.loadtxt(data_path_train, dtype=np.float32, delimiter=',', skiprows=1)
arr_test = np.loadtxt(data_path_test, dtype=np.float32, delimiter=',', skiprows=1)

tensor_train = torch.from_numpy(arr_train)
tensor_test = torch.from_numpy(arr_test)

# dataset_train = [(row[1:].to(torch.float64).reshape(28, 28).unsqueeze(0), row[0].item()) for row in tensor_train]
# dataset_train = [(row[1:].reshape(28, 28).unsqueeze(0), row[0].to(torch.int).item()) for row in tensor_train]
dataset_train = [(row[1:].reshape(28, 28).unsqueeze(0), row[0].item()) for row in tensor_train]
dataset_test = [(row[1:].reshape(28, 28).unsqueeze(0), row[0].item()) for row in tensor_test]

In [31]:
dataset_train[0][1]

3.0

In [32]:
# imgs = torch.stack([img for (img, label) in dataset_train], dim = 3)
# mean_deviation = imgs.view(1, -1).mean(dim=1).item()
# standard_deviation = imgs.view(1, -1).std(dim=1).item()
# # print(dataset_train.shape())
# dataset_train = dataset_train/255.0
# dataset_test = dataset_test/255.0

dataset_train = [(img/255.0, int(label)) for (img, label) in dataset_train]
dataset_test = [(img/255.0, int(label)) for (img, label) in dataset_test]

In [33]:
img, lbl = dataset_train[23]
img

# from matplotlib import pyplot as plt
# plt.imshow(img.permute(1,2,0))

tensor([[[0.3333, 0.3529, 0.3804, 0.4039, 0.4667, 0.5098, 0.5412, 0.5765,
          0.5961, 0.6157, 0.6314, 0.6431, 0.6392, 0.6549, 0.6706, 0.6902,
          0.6980, 0.7059, 0.7137, 0.7216, 0.7255, 0.7373, 0.7412, 0.7490,
          0.7569, 0.7569, 0.7569, 0.7569],
         [0.3373, 0.3529, 0.3765, 0.4078, 0.4784, 0.5137, 0.5490, 0.5765,
          0.5961, 0.6157, 0.6353, 0.6510, 0.6863, 0.6588, 0.6941, 0.6980,
          0.6784, 0.6941, 0.7333, 0.7216, 0.7373, 0.7490, 0.7529, 0.7569,
          0.7608, 0.7647, 0.7647, 0.7647],
         [0.3412, 0.3608, 0.3843, 0.4157, 0.4824, 0.5216, 0.5569, 0.5804,
          0.6039, 0.6314, 0.6157, 0.7412, 0.8196, 0.6980, 0.7333, 0.7804,
          0.6588, 0.4196, 0.6588, 0.7529, 0.7373, 0.7529, 0.7569, 0.7608,
          0.7608, 0.7647, 0.7686, 0.7686],
         [0.3412, 0.3647, 0.3922, 0.4196, 0.4902, 0.5255, 0.5647, 0.5922,
          0.6118, 0.6471, 0.6196, 0.8235, 0.7294, 0.5647, 0.6745, 0.7059,
          0.6314, 0.3686, 0.4667, 0.7882, 0.7373, 0.7529,

In [34]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset_test, batch_size=64, shuffle=False)

print(len(train_loader), len(val_loader))
print(len([(imgs, labels) for imgs, labels in train_loader][0][0]),
      len([(imgs, labels) for imgs, labels in val_loader][0][0]))

429 113
64 64


In [91]:
import torch.nn as nn
import torch.nn.functional as F

class NetDropout(nn.Module):
    def __init__(self, n = 32):
        super().__init__()
        self.n = n
        
        self.conv1 = nn.Conv2d(1, self.n, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(self.n, (self.n//2), kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.conv3 = nn.Conv2d((self.n//2), (self.n//2), kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout2d(p=0.3)

        self.fc1 = nn.Linear((self.n//2)*3*3, 32)
        self.fc2 = nn.Linear(32, 25)
        
        self.lsftmx = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)

        out = out.view(-1, (self.n//2)*3*3)
        out = torch.tanh(self.fc1(out))
        out = self.lsftmx(self.fc2(out))
        return out
    
# class NetVarSize(nn.Module):
#     def __init__(self, n = 32):
#         super().__init__()
#         self.n = n
        
#         self.conv1 = nn.Conv2d(1, self.n, kernel_size=3, padding=1)
#         self.conv1_dropout = nn.Dropout2d(p=0.3)
#         self.conv2 = nn.Conv2d(self.n, (self.n//2), kernel_size=3, padding=1)
#         self.conv2_dropout = nn.Dropout2d(p=0.3)
#         self.conv3 = nn.Conv2d((self.n//2), (self.n//2), kernel_size=3, padding=1)
#         self.conv3_dropout = nn.Dropout2d(p=0.3)

#         self.fc1 = nn.Linear((self.n//2)*3*3, 32)
#         self.fc2 = nn.Linear(32, 25)
        
#         self.lsftmx = nn.LogSoftmax(dim=1)

#     def forward(self, x):
#         out = F.adaptive_max_pool2d(torch.tanh(self.conv1(x)), (2, 2))
#         out = self.conv1_dropout(out)
#         out = F.adaptive_max_pool2d(torch.tanh(self.conv2(out)), (2, 2))
#         out = self.conv2_dropout(out)
#         out = F.adaptive_max_pool2d(torch.tanh(self.conv3(out)), (2, 2))
#         out = self.conv3_dropout(out)

#         out = out.view(out.size(0), -1)
#         out = torch.tanh(self.fc1(out))
#         out = self.fc2(out)
#         out = self.lsftmx(out)
#         return out

In [36]:
def training_loop(n_epochs, model, loss_fn, optimizer, train_loader, device):
    for epoch in range(1, n_epochs + 1):
        for (imgs, labels) in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)

            outs = model(imgs)
            loss = loss_fn(outs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 10 == 0 or epoch <= 3:
            print("epoch=%d loss=%f" %(epoch, loss)) # loss of last batch

    total = 0
    correct = 0
    for imgs, labels in train_loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)

        outs = model(imgs)
        max_vals, max_classes = outs.max(dim=1)

        total += imgs.shape[0]
        correct += (max_classes == labels).sum()

    print("training accuracy =%f" %(correct/total))

In [37]:
def validate(model, loader, device):
    total = 0
    correct = 0
    for (imgs, lbls) in loader:
        imgs = imgs.to(device=device)
        lbls = lbls.to(device=device)

        outs = model(imgs)
        max_vals, max_indexes = outs.max(dim=1)
        
        correct += ((max_indexes == lbls).sum())
        total += imgs.shape[0]
    print("accuracy %f" %(correct/total))

In [38]:
# import torch.optim as optim
# import os

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model = NetDropout().to(device=device)

# training_loop(
#     n_epochs=100,
#     model=model,
#     train_loader=train_loader,
#     loss_fn=nn.NLLLoss(),
#     optimizer=optim.SGD(model.parameters(), lr=0.01),
#     device=device
# )

# validate(model, val_loader, device=device)

epoch=1 loss=3.209289
epoch=2 loss=3.179486
epoch=3 loss=3.206956
epoch=10 loss=2.290270
epoch=20 loss=1.206905
epoch=30 loss=0.509000
epoch=40 loss=0.506229
epoch=50 loss=0.274397
epoch=60 loss=0.270535
epoch=70 loss=0.195263
epoch=80 loss=0.291559
epoch=90 loss=0.114831
epoch=100 loss=0.131022
training accuracy =0.955746
accuracy 0.850390


In [42]:
# torch.save(model.state_dict(), "./model.t")

<All keys matched successfully>

In [92]:
import torch.optim as optim
import os

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model2 = NetVarSize().to(device=device)

training_loop(
    n_epochs=100,
    model=model2,
    train_loader=train_loader,
    loss_fn=nn.NLLLoss(),
    optimizer=optim.SGD(model2.parameters(), lr=0.01),
    device=device
)

validate(model2, val_loader, device=device)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x64 and 144x32)

In [ ]:

torch.save(model2.state_dict(), "./model2.t")